In [3]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [4]:
from traj_xai.src import SimpleTransformerModel, TrajFormerModel

/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [5]:
from pactus import Dataset

## Segmentation


In [6]:
from traj_xai.src import (
    mdl_segmentation,
    sliding_window_segmentation,
)

## Pertubation


In [7]:
from traj_xai.src import (
    scaling_perturbation,
    rotation_perturbation,
)

## Evaluation


## Experiment with all trajectories


In [ ]:
# LSTM Model
SEED = 0

# datasets = [
#     # Dataset.geolife(),
#     Dataset.animals(),
#     # Dataset.hurdat2(),
#     Dataset.uci_characters(),
#     Dataset.cma_bst(), 1k
#     # Dataset.mnist_stroke(), 1k
#     # Dataset.uci_pen_digits(), 1k
#     # Dataset.uci_gotrack(),
#     Dataset.uci_movement_libras(),
# ]
# train, test = dataset.mnist_stroke(.8, random_state=SEED)

# Load dataset
dataset = Dataset.animals()

# Split data into train and test subsets
train, test = dataset.split(0.8, random_state=SEED)
# train, test = dataset.cut(1000)

10:44:42 [INFO] Creating dataset folder for animals
10:44:42 [INFO] Downloading animals dataset


10:44:53 [INFO] Extracting animals dataset


[==================================================] 100.00%   Size: 5.85 MB


In [ ]:
# train.classes

['Elk', 'Cattle', 'Deer']

### DL Model


In [10]:
max_length = 1000
embedding_size = 64

# Build and train the model
model = TrajFormerModel(
    c_out=len(dataset.classes),
)

# Train the model on the train dataset
model.train(train, dataset, epochs=10, batch_size=64, checkpoint=None)

# Evaluate the model on a test dataset
evaluation = model.evaluate(test)

# Print the evaluation
evaluation.show()

10:45:01 [INFO] Epoch 10/10, Loss: 3.8908
10:45:01 [INFO] Trained TrajFormer model with 202 samples



General statistics:

Accuracy: 0.569
F1-score: 0.357
Mean precision: 0.360
Mean recall: 0.409

Confusion matrix:

Cattle  Deer    Elk     precision
26.67   30.0    3.85    50.0    
0.0     0.0     0.0     0.0     
73.33   70.0    96.15   58.14   
--------------------------------
26.67   0.0     96.15   


### ML Model


In [11]:
# # ML Model
# SEED = 0

# # datasets = [
# #     # Dataset.geolife(),
# #     Dataset.animals(),
# #     # Dataset.hurdat2(), -> RF cant work
# #     Dataset.cma_bst(),
# #     # Dataset.mnist_stroke(), -> okay
# #     # Dataset.uci_pen_digits(),
# #     # Dataset.uci_gotrack(),
# #     Dataset.uci_characters(),
# #     Dataset.uci_movement_libras(),-> okay
# # ]

# # Load dataset
# dataset = Dataset.geolife()

# # Split data into train and test subsets
# # train, test = dataset.split(.995, random_state=SEED)

# train, test = dataset.filter(
#         lambda traj, label: len(traj) >= 5
#         and traj.r.delta.norm.sum() > 0
#         and dataset.label_counts[label] > 5
#     ).split(
#         train_size=0.985,
#         random_state=SEED,
#     )

# # Build and train the model
# featurizer = featurizers.UniversalFeaturizer()
# model = XGBoostModel(featurizer=featurizer)
# # model = SVMModel(
# #     featurizer=featurizer,
# #     random_state=SEED,
# # )
# # DecisionTreeModel(
# #     featurizer=featurizer,
# #     max_depth=7,
# #     random_state=SEED,
# # # )


# # Train the model on the train dataset
# model.train(train, 5) # 5: CV

# # Evaluate the model on a test dataset
# evaluation = model.evaluate(test)

# # Print the evaluation
# evaluation.show()

### Experiment


In [12]:
from traj_xai.src import run_experiments

In [13]:
# segment_func = [rdp_segmentation]
# perturbation_func = [gaussian_perturbation]
segment_func = [mdl_segmentation, sliding_window_segmentation]
perturbation_func = [scaling_perturbation, rotation_perturbation]

run_experiments(
    dataset,
    segment_func,
    perturbation_func,
    model,
)

Running experiment with mdl_segmentation and scaling_perturbation
[DEBUG] Pre-explain segments: 12
[DEBUG] About to call model.predict() in explain()
[DEBUG] Attempting standard predict call
[DEBUG] model.predict() result type: <class 'numpy.ndarray'>
[DEBUG] model.predict() returned ndarray of shape (1024, 3)
[DEBUG] preds type: <class 'numpy.ndarray'>, value: [[0.09120873 0.29575738 0.61303395]
 [0.09112807 0.29563174 0.6132402 ]
 [0.09120873 0.29575738 0.61303395]
 ...
 [0.09112807 0.29563174 0.6132402 ]
 [0.09120873 0.29575738 0.61303395]
 [0.09120873 0.29575738 0.61303395]]
[DEBUG] pred_labels: [np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np

KeyboardInterrupt: 